In [ ]:
import matplotlib.pyplot as plt
import cmasher as cmr
import numpy as np
import h5py
import pickle
import requests
import os.path
from astropy.cosmology import LambdaCDM
from unyt import Myr, K
from synthesizer.particle import Stars, Gas, Galaxy
from synthesizer.grid import Grid
from synthesizer.emission_models import IncidentEmission, PacmanEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Greybody

### Parameters

In [ ]:
# ADD YOUR OWN API KEY HERE
api_key = "c37483fea84b70f4ad9f777c38357ea0"

# this is a single galaxy that has been manually identified as interesting
id = '145492'
redshift = 2.0  # note: this is not the exact redshift of this snapshot, this is extracted later. 

# flag whether to calculate line-of-sight dust
calculate_los_dust = False

# the optical depth to use otherwise
if not calculate_los_dust:
    tau_v = 0.5

### Get properties of the simulation

In [ ]:
headers = {"api-key":api_key}

def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically
    return r


In [ ]:

# URL for the specific simulation
baseUrl = 'http://www.tng-project.org/api/TNG50-1/'

# grab the details of this simulation
simulation = get(baseUrl)

# print the information about the simulation
for k, v in simulation.items():
    print(f'{k}: {v}')

### Define cosmology

In [ ]:
# extract the cosmological parameters for this simulation
h = simulation['hubble']
Om0 = simulation['omega_0']
Ode0 = simulation['omega_L']
H0 = h * 100 
Ob0 = simulation['omega_B']

# create an astropy cosmology object
cosmo = LambdaCDM(Om0=Om0, Ode0=Ode0, H0=H0, Ob0=Ob0)

In [ ]:
# create a dictionary of snapshots and redshifts (useful later)
snapshots = get(simulation['snapshots'])
snapshot_redshifts = {snapshot['number']: snapshot['redshift'] for snapshot in snapshots}
print(snapshot_redshifts)

### Download data

In [ ]:
def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open('data/'+filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [ ]:
# the quantities that we want to grab: the coordinates, metallicity, age, and initial mass of all the star particles
params = {
    'stars':'Coordinates,GFM_Metallicity,GFM_StellarFormationTime,GFM_InitialMass,Masses',
    'gas': 'Coordinates,GFM_Metallicity,SubfindHsml,Masses'
    }

# the data url for the object of interest
url = f"{baseUrl}/snapshots/z={str(redshift)}/subhalos/{str(id)}"

# Download the subhalo properties for the specific galaxy and save them as pickle file
subhalo_properties_filename = f'data/subhalo_properties_{id}.pck'
if not os.path.isfile(subhalo_properties_filename):
    subhalo_properties = get(url) # get json response of subhalo properties
    pickle.dump(subhalo_properties, open(subhalo_properties_filename,'wb'))

# Download the HDF5 cutout for the galaxy if it hasn't already been downloaded
cutout_filename = f'data/cutout_{id}.hdf5'
if not os.path.isfile(cutout_filename):
    get(url + "/cutout.hdf5", params) # get and save HDF5 cutout file

Print sub-halo properties and determine the exact snapshot redshift

In [ ]:
subhalo_properties = pickle.load(open(f'data/subhalo_properties_{id}.pck','rb'))
for k,v in subhalo_properties.items():
    print(f'{k}: {v}')

snapshot_redshift = snapshot_redshifts[subhalo_properties['snap']]
print(f'snapshot redshift: {snapshot_redshift}')

Explore HDF5 file

In [ ]:
with h5py.File(cutout_filename) as f:
    f.visit(print)

### Creating a Galaxy object

After downloading the data we need to create a synthesizer galaxy object.


In [ ]:

from unyt import Msun, Mpc

# define the centre of the galaxy
centre = np.array([subhalo_properties['pos_x'],
                   subhalo_properties['pos_y'],
                   subhalo_properties['pos_z'],
                   ]) * Mpc / (1 + snapshot_redshift) / h / 1000

print(centre)

with h5py.File(cutout_filename) as f:

    # GFM_StellarFormationTime is the scale factor when the  
    formation_scale_factor = f['PartType4']['GFM_StellarFormationTime'][()]
    formation_redshift = 1/formation_scale_factor - 1.0
    
    # only select star particles that make sense
    s = formation_redshift >= snapshot_redshift
    formation_redshift = formation_redshift[s]

    # calculate the ages of star particle and record as unyt quantity
    ages = (cosmo.age(z=snapshot_redshift)-cosmo.age(formation_redshift)).to('Myr').value * Myr

    # convert units of initial masses
    initial_masses = f['PartType4']['GFM_InitialMass'][s] * Msun * 1E10 / h 

    # convert units of initial masses
    current_masses = f['PartType4']['Masses'][s] * Msun * 1E10 / h 

    # define the coorindates of the star particles 
    x = f['PartType4']['Coordinates'][s,0] 
    y = f['PartType4']['Coordinates'][s,1] 
    z = f['PartType4']['Coordinates'][s,2] 

    # convert coordinates to physical and change the units
    coordinates = np.array([x,y,z]).T / (1+snapshot_redshift) / h / 1000 * Mpc

    # create a stars object
    stars = Stars(
        ages=ages,
        initial_masses=initial_masses,
        current_masses=current_masses,
        metallicities=f['PartType4']['GFM_Metallicity'][s],
        coordinates=coordinates,
        centre=centre,
        )
    
    # define the coorindates of the gas particles 
    x = f['PartType0']['Coordinates'][:,0] 
    y = f['PartType0']['Coordinates'][:,1] 
    z = f['PartType0']['Coordinates'][:,2] 

    # convert coordinates to physical and change the units
    coordinates = np.array([x,y,z]).T  / (1 + snapshot_redshift) / h / 1000 * Mpc


    # create a gas object
    gas = Gas(
        masses=f['PartType0']['Masses'][:] * 1E10 / h * Msun,
        metallicities=f['PartType0']['GFM_Metallicity'][:],
        smoothing_lengths=f['PartType0']['SubfindHsml'][:] / (1 + snapshot_redshift) / h / 1000 * Mpc,
        coordinates=coordinates,
        centre=centre,
        )


# initialise the galaxy object
galaxy = Galaxy(stars=stars, gas=gas, centre=centre, redshift=snapshot_redshift)

### Show SFZH

In [ ]:
metallicities = galaxy.stars.metallicities
ages = galaxy.stars.ages
initial_masses = galaxy.stars._initial_masses

cmap = cmr.sapphire
age_range = [0., 3000.]
log10metallicity_range = [-3.5, -1.]
nbins = 40

fig = plt.figure(figsize=(3.5, 3.5))

bottom = 0.15
height = 0.6
hsize = 0.15
left = 0.15
width = 0.6

ax = fig.add_axes((left, bottom, width, height))
axx = fig.add_axes((left, bottom+height, width, hsize))
axy = fig.add_axes((left+width, bottom, hsize, height))

hist, xedges, yedges = np.histogram2d(ages.to('Myr').value, 
                                      np.log10(metallicities), 
                                      bins=[nbins,nbins], 
                                      range=[age_range, log10metallicity_range],
                                      weights=initial_masses)

axx.hist(ages.to('Myr').value, bins=nbins, range=age_range, weights=initial_masses,color='0.5')
axy.hist(np.log10(metallicities), bins=nbins, range=log10metallicity_range, weights=initial_masses, orientation='horizontal',color='0.5')

ax.imshow(hist.T, 
           origin='lower', 
           interpolation='nearest', 
           extent=[*age_range, *log10metallicity_range],
           aspect='auto',
           cmap=cmap)


axx.set_xlim(age_range)
axx.set_yticklabels([])
axx.xaxis.set_ticks_position('top')

axy.set_ylim(log10metallicity_range)
axy.set_xticklabels([])
axy.yaxis.set_ticks_position('right')

ax.set_xlabel(r'$\rm age/Myr$')
ax.set_ylabel(r'$\rm log_{10}(metallicity)$')

fig.savefig('figs/tng-age_metallicity.pdf')
fig.show()

### LOS dust

Calculate the line-of-sight surface density of dust to each star particle in the galaxy.

This should take up-to 20 minutes.



In [ ]:

if calculate_los_dust:

    from synthesizer.kernel_functions import Kernel

    kappa = 0.3
    kernel = Kernel().get_kernel()
    galaxy.calculate_los_tau_v(kappa=kappa, kernel=kernel, force_loop=True)

else:

    galaxy.stars.tau_v = tau_v * np.ones(len(galaxy.stars.ages))


Plot the distribution of $\tau_V$ values:

In [ ]:
tauv = galaxy.stars.tau_v
tauv = tauv[tauv>0.0]

plt.hist(np.log10(tauv), bins=100)
plt.show()

### Save

We now use pickle to save the Galaxy object for use in other notebooks.

In [ ]:
import pickle

pickle.dump(galaxy, open(f'data/galaxy_{id}.pck','wb'))